In [1]:
import os

In [2]:
%pwd

'/home/rgukt/Desktop/NLPproj/Text_Summerization-NLP/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/rgukt/Desktop/NLPproj/Text_Summerization-NLP'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [13]:
from textSummerizer.constants import *
from textSummerizer.utils.common import read_yaml,create_directories

In [14]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            params_filepath= PARAMS_FILE_PATH):

            self.config=read_yaml(config_filepath)
            self.params=read_yaml(params_filepath)
    
            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
          config=self.config.data_ingestion
          create_directories([config.root_dir])


          data_ingestion_config=DataIngestionConfig(
                root_dir=config.root_dir,
                source_URL=config.source_URL,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir
          )

          return data_ingestion_config

In [16]:
import os
import urllib.request as request
import zipfile
from src.textSummerizer.logging import logger
from textSummerizer.utils.common import get_size

In [18]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded ! with following info {headers}")
        else:
            logger.info(f"file alredy exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [19]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-09-15 16:43:25,372: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-09-15 16:43:25,802: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-15 16:43:25,882: INFO: common: created directory at: artifacts]
[2025-09-15 16:43:25,904: INFO: common: created directory at: artifacts/data_ingestion]


URLError: <urlopen error [Errno -3] Temporary failure in name resolution>